<a href="https://colab.research.google.com/github/macsyd/GPU-Computing/blob/main/Assignment2/csc485b_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ktcox7my
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ktcox7my
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp5squyb0j".


In [2]:
%%cuda_group_save -g "source" -n "data_types.h"
/**
 * A collection of commonly used data types throughout this project.
 */
#pragma once

#include <iostream> // for std::ostream
#include <vector>

namespace csc485b{
namespace a2{

using node_t = int;
using edge_t = int2;

using edge_list_t = std::vector< edge_t >;
using node_list_t = std::vector< node_t >;

} // namespace a2
} // namespace csc485b


In [3]:
%%cuda_group_save -g "source" -n "cuda_common.h"
/**
 * Standard macros that can be useful for error checking.
 * https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html
 */
#pragma once

#include <cuda.h>

#define CUDA_CALL(exp)                                       \
    do {                                                     \
        cudaError res = (exp);                               \
        if(res != cudaSuccess) {                             \
            printf("Error at %s:%d\n %s\n",                  \
                __FILE__,__LINE__, cudaGetErrorString(res)); \
           exit(EXIT_FAILURE);                               \
        }                                                    \
    } while(0)

#define CHECK_ERROR(msg)                                             \
    do {                                                             \
        cudaError_t err = cudaGetLastError();                        \
        if(cudaSuccess != err) {                                     \
            printf("Error (%s) at %s:%d\n %s\n",                     \
                (msg), __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE);                                      \
        }                                                            \
    } while (0)

In [4]:
%%cuda_group_save -g "source" -n "data_generator.h"
/**
 * Functions for generating random input data with a fixed seed
 */
#pragma once

#include <cassert>  // for assert()
#include <cstddef>  // std::size_t type
#include <random>   // for std::mt19937, std::uniform_int_distribution
#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a2 {

/**
 * Generates and returns a vector of random edges
 * for a graph `G=(V,E)` with `n=|V|=n` and expected `m=|E|`.
 * Referred to as an Erdős-Rényi graph.
 *
 * @see https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.fast_gnp_random_graph.html#networkx.generators.random_graphs.fast_gnp_random_graph
 */
edge_list_t generate_graph( std::size_t n, std::size_t m )
{
    assert( "At most n(n-1) edges in a simple graph" && m < n * ( n - 1 ) );

    int const probability = ( 100 * m ) / ( n * ( n - 1 ) );

    // for details of random number generation, see:
    // https://en.cppreference.com/w/cpp/numeric/random/uniform_int_distribution
    std::size_t random_seed = 20241008;  // use magic seed
    std::mt19937 rng( random_seed );     // use mersenne twister generator
    std::uniform_int_distribution<> distrib(0, 100);

    edge_list_t random_edges;
    random_edges.reserve( 2 * m );

    for( node_t u = 0; u < n; ++u )
    {
        for( node_t v = u + 1; v < n; ++v )
        {
            auto const dice_roll = distrib( rng );
            if( dice_roll <= probability )
            {
                random_edges.push_back( make_int2( u, v ) );
                random_edges.push_back( make_int2( v, u ) );
            }
        }
    }

    random_edges.resize( random_edges.size() );


    return random_edges;
}

 /**
  * Generate a simple graph with a known adjacencyMatrix and Two Hop for testing
 */
edge_list_t generate_known_graph( std::size_t n, std::size_t m )
{
    assert( "At most n(n-1) edges in a simple graph" && m < n * ( n - 1 ) );

    edge_list_t random_edges;
    // Node 0 connections
    random_edges.push_back( make_int2(0, 1) );
    random_edges.push_back( make_int2(1, 0) );
    random_edges.push_back( make_int2(0, 2) );
    random_edges.push_back( make_int2(2, 0) );

    // Node 1 connections
    random_edges.push_back( make_int2(1, 3) );
    random_edges.push_back( make_int2(3, 1) );

    // Node 2 connections
    random_edges.push_back( make_int2(2, 3) );
    random_edges.push_back( make_int2(3, 2) );

    random_edges.resize( random_edges.size() );

    return random_edges;
}

} // namespace a2
} // namespace csc485b


In [5]:
%%cuda_group_save -g "source" -n "dense_graph.h"
/**
 * The file in which you will implement your DenseGraph GPU solutions!
 */

#include <cstddef>  // std::size_t type

#include "cuda_common.h"
#include "data_types.h"

namespace csc485b {
namespace a2      {

/**
 * A DenseGraph is optimised for a graph in which the number of edges
 * is close to n(n-1). It is represented using an adjacency matrix.
 */
struct DenseGraph
{
  std::size_t n; /**< Number of nodes in the graph. */
  node_t * adjacencyMatrix; /** Pointer to an n x n adj. matrix */
  node_t * twoHopMatrix; /** Pointer to n x n two-hop adj matrix */

  /** Returns number of cells in the adjacency matrix. */
  __device__ __host__ __forceinline__
  std::size_t matrix_size() const { return n * n; }
};


namespace gpu {


/**
 * Constructs a DenseGraph from an input edge list of m edges.
 *
 * @pre The pointers in DenseGraph g have already been allocated.
 */
__global__
void build_graph( DenseGraph g, edge_t const * edge_list, std::size_t m )
{
    // IMPLEMENT ME!

    // set indices (i,j) and (j,i) to 1 in the adjacency matrix (g.adajcencyMatrix[i][j] and  g.adajcencyMatrix[j][i]) for each edge (i,j) in edge_list
    // every other entry gets set to 0 (already happens by default)x
    int const th_id = blockIdx.x * blockDim.x + threadIdx.x;

    if (th_id < m) {
        edge_t my_edge = edge_list[th_id];
        int i = my_edge.x;
        int j = my_edge.y;

        g.adjacencyMatrix[i * (g.n) + j] = 1;
        //g.adjacencyMatrix[j * (g.n) + i] = 1; //not needed because edg (i,j) and (j,i) both appear in edge list
    }

    return;
}

/**
  * Repopulates the adjacency matrix as a new graph that represents
  * the two-hop neighbourhood of input graph g
  */
__global__
void two_hop_reachability( DenseGraph g )
{
    int const th_id = blockIdx.x * blockDim.x + threadIdx.x;
    if (th_id < g.n * g.n)
    {
        int row = th_id / g.n;
        int col = th_id % g.n;
        int temp_sum = 0;

        for(int i = 0; i < g.n; i++)
        {
          temp_sum += g.adjacencyMatrix[row * g.n + i] * g.adjacencyMatrix[i * g.n + col];
        }
        // Clamp to [0,1]-> if greater than 0, set to 1, else set to 0
        node_t clamped_val = (temp_sum > 0) ? 1 : 0;

        if (row == col) {
            //ignore diagonals
            clamped_val = 0;
        }
        g.twoHopMatrix[th_id] = clamped_val;
    }
    return;
}

} // namespace gpu
} // namespace a2
} // namespace csc485b

In [6]:
%%cuda_group_save -g "source" -n "sparse_graph.h"
/**
 * The file in which you will implement your SparseGraph GPU solutions!
 */

#include <cstddef>  // std::size_t type

#include "cuda_common.h"
#include "data_types.h"

namespace csc485b {
namespace a2      {

/**
 * A SparseGraph is optimised for a graph in which the number of edges
 * is close to cn, for a small constanct c. It is represented in CSR format.
 */
struct SparseGraph
{
  std::size_t n; /**< Number of nodes in the graph. */
  std::size_t m; /**< Number of edges in the graph. */
  node_t * neighbours_start_at; /** Pointer to an n=|V| offset array */
  node_t * neighbours; /** Pointer to an m=|E| array of edge destinations */
};


namespace gpu {
/**
 * Helper function to build the neighbours_start_at array from
 * the sum of the edges between a given node
*/
__global__
void prefix_sum(SparseGraph g, edge_t const * edge_list, std::size_t m) {
    // run a prefix sum to get running counts of number of threads' neighbours
    // create shared mem for double buffering
    int const th_id = blockIdx.x * blockDim.x + threadIdx.x;
    int constexpr shared_memory_size = 1024;
    __shared__ node_t smem[ shared_memory_size ];
    if(th_id < g.n) {
        smem[th_id] = g.neighbours_start_at[th_id];
    }
    __syncthreads();

    for(int stride = 1; stride < g.n; stride++) { //(g.n >> 1) = log2f(g.n)
        if(th_id < g.n) {
            if(th_id > stride) {
                smem[th_id] += g.neighbours_start_at[th_id - stride];
            }
        }
    }
    __syncthreads();
    if(th_id < g.n) {
        g.neighbours_start_at[th_id] = smem[th_id];
    }
}
/**
 * Helper function to build the neighbours_start_at array from
 * the sum of the edges between a given node
*/
__global__
void merge_sums(SparseGraph g, int stride) {
    // add previous prefix sum blocks so whole neighbours_start_at array has prefix sum
    // create shared memory for double buffering
    int const th_id = blockIdx.x * blockDim.x + threadIdx.x;
    int constexpr shared_memory_size = 1024;
    __shared__ node_t smem[ shared_memory_size ];
    if(th_id < g.n) {
        smem[th_id] = g.neighbours_start_at[th_id];
    }
    __syncthreads();

    smem[th_id] += g.neighbours_start_at[th_id - g.n*stride + (g.n - (th_id % g.n) - 1)];
    __syncthreads();

    if(th_id < g.n) {
        g.neighbours_start_at[th_id] = smem[th_id];
    }
}

__global__
void fill_array(SparseGraph g, edge_t const * edge_list, std::size_t m)
{
    int const th_id = blockIdx.x * blockDim.x + threadIdx.x;
    if (th_id < g.n) {
        g.neighbours_start_at[th_id] = 0;
    }

    if (th_id < m) {
        edge_t my_edge = edge_list[th_id];
        int i = my_edge.x;
        if (i+1 < m) atomicAdd(&g.neighbours_start_at[i+1], 1);
    }
    __syncthreads();
}
/**
 * Constructs a SparseGraph from an input edge list of m edges.
 *
 * @pre The pointers in SparseGraph g have already been allocated.
 */
void build_graph( SparseGraph g, edge_t const * edge_list, std::size_t m, std::size_t const num_blocks, std::size_t const threads_per_block )
{
    // IMPLEMENT ME!

    // fill neighbours_start_at with number of neighbours
    fill_array<<< num_blocks, threads_per_block >>>(g, edge_list, m);

    // device function (above) for prefix sum
    prefix_sum<<< num_blocks, threads_per_block >>>(g, edge_list, m);

    //for mulitple thread blocks:
    //merge sums in increasing strides
    int thread_block_count = g.n/8; // num thread blocks - should this be hard coded or passed in?
    for(int stride = 1; stride < thread_block_count/2; stride <<= 1) {
        merge_sums<<< 1, num_blocks >>>(g, stride);
        cudaDeviceSynchronize();
    }

    // use neighbours_start_at entries when filling in the neighbours array to know which "chunk" to put the neighbours in

}

/**
  * Repopulates the adjacency lists as a new graph that represents
  * the two-hop neighbourhood of input graph g
  */
__global__
void two_hop_reachability( SparseGraph g )
{
    // IMPLEMENT ME!
    // algorithm unknown
    return;
}

} // namespace gpu
} // namespace a2
} // namespace csc485b

In [7]:
%%cuda_group_save -g "source" -n "main.cu"
/**
 * Driver for the benchmark comparison. Generates random data,
 * runs the CPU baseline, and then runs your code.
 */

#include <chrono>   // for timing
#include <iostream> // std::cout, std::endl
#include <iterator> // std::ostream_iterator
#include <vector>

#include "dense_graph.h"
#include "sparse_graph.h"

#include "data_generator.h"
#include "data_types.h"

/**
 * Runs timing tests on a CUDA graph implementation.
 * Consists of independently constructing the graph and then
 * modifying it to its two-hop neighbourhood.

 * Allocates space for a dense graph and then runs the test code on it.
 */
void run_dense( csc485b::a2::edge_t const * d_edges, std::size_t n, std::size_t m )
{
    using namespace csc485b;

    // allocate device DenseGraph
    a2::node_t * d_matrix;
    a2::node_t * d_twoHopMatrix;
    cudaMalloc( (void**)&d_matrix, sizeof( a2::node_t ) * n * n );
    cudaMalloc( (void**)&d_twoHopMatrix, sizeof( a2::node_t ) * n * n );
    a2::DenseGraph d_dg{ n, d_matrix, d_twoHopMatrix };
    printf("Producing Dense Graph and Two Hop Reachability:\n");
    //run( d_dg, d_edges, m );

    cudaDeviceSynchronize();
    auto const build_start = std::chrono::high_resolution_clock::now();

    //kernel launch configs
    std::size_t const threads_per_block = 2;
    //std::size_t const num_blocks =  m / threads_per_block + 1; // use one thread per edge
    std::size_t const num_blocks =  8;
    // this code doesn't work yet!
    csc485b::a2::gpu::build_graph<<< num_blocks, threads_per_block >>>( d_dg, d_edges, m );

    cudaDeviceSynchronize();
    auto const reachability_start = std::chrono::high_resolution_clock::now();

    // neither does this!
    csc485b::a2::gpu::two_hop_reachability<<< num_blocks, threads_per_block >>>( d_dg );

    cudaDeviceSynchronize();
    auto const end = std::chrono::high_resolution_clock::now();

    std::cout << "Build time: "
              << std::chrono::duration_cast<std::chrono::microseconds>(reachability_start - build_start).count()
              << " us"
              << std::endl;

    std::cout << "Reachability time: "
              << std::chrono::duration_cast<std::chrono::microseconds>(end - reachability_start).count()
              << " us"
              << std::endl;

    // check output?
    std::vector<a2::node_t> host_adjacency_matrix(d_dg.matrix_size(), 0);
  std::vector<a2::node_t> host_twoHop_matrix(d_dg.matrix_size(), 0);
    a2::DenseGraph dg{ n, host_adjacency_matrix.data(), host_twoHop_matrix.data() };
    cudaMemcpy( dg.adjacencyMatrix, d_dg.adjacencyMatrix, sizeof( a2::node_t ) * d_dg.matrix_size(), cudaMemcpyDeviceToHost );
    cudaMemcpy( dg.twoHopMatrix, d_dg.twoHopMatrix, sizeof( a2::node_t ) * d_dg.matrix_size(), cudaMemcpyDeviceToHost );
    std::cout << "Dense Adjacency Matrix: ";
    std::copy( host_adjacency_matrix.cbegin(), host_adjacency_matrix.cend(), std::ostream_iterator< a2::node_t >( std::cout, " " ) );
    std::cout << std::endl;
    std::cout << "Two Hop Matrix: ";
    std::copy( host_twoHop_matrix.cbegin(), host_twoHop_matrix.cend(), std::ostream_iterator< a2::node_t >( std::cout, " " ) );
    std::cout << "\n_____________________________________________________________________________________________________________" << std::endl;

    // clean up
    cudaFree( d_matrix );
    cudaFree( d_twoHopMatrix);
}

/**
 * Allocates space for a sparse graph and then runs the test code on it.
 */
void run_sparse( csc485b::a2::edge_t const * d_edges, std::size_t n, std::size_t m )
{
    using namespace csc485b;

    // allocate device SparseGraph
    a2::node_t * d_offsets, * d_neighbours;
    cudaMalloc( (void**)&d_offsets,    sizeof( a2::node_t ) * n );
    cudaMalloc( (void**)&d_neighbours, sizeof( a2::node_t ) * m );
    a2::SparseGraph d_sg{ n, m, d_offsets, d_neighbours };
    printf("\nProducing Sparse Graph and Two Hop Reachability:\n");
    //run( d_sg, d_edges, m );

    cudaDeviceSynchronize();
    auto const build_start = std::chrono::high_resolution_clock::now();

    //kernel launch configs
    std::size_t const threads_per_block = 4;
    std::size_t const num_blocks =  m / threads_per_block + 1; // use one thread per edge

    // this code doesn't work yet!
    csc485b::a2::gpu::build_graph(d_sg, d_edges, m, num_blocks, threads_per_block);

    cudaDeviceSynchronize();
    auto const reachability_start = std::chrono::high_resolution_clock::now();

    // neither does this!
    csc485b::a2::gpu::two_hop_reachability<<< num_blocks, threads_per_block >>>( d_sg );

    cudaDeviceSynchronize();
    auto const end = std::chrono::high_resolution_clock::now();

    std::cout << "Build time: "
              << std::chrono::duration_cast<std::chrono::microseconds>(reachability_start - build_start).count()
              << " us"
              << std::endl;

    std::cout << "Reachability time: "
              << std::chrono::duration_cast<std::chrono::microseconds>(end - reachability_start).count()
              << " us"
              << std::endl;

    // check output?
    std::vector< a2::node_t > host_matrix_sparse( d_sg.m );
    std::vector< a2::node_t > host_matrix_2( d_sg.n );
    a2::SparseGraph sg{ n, m, host_matrix_2.data(), host_matrix_sparse.data() };
    cudaMemcpy( sg.neighbours_start_at, d_sg.neighbours_start_at, sizeof( a2::node_t ) * d_sg.n, cudaMemcpyDeviceToHost );
    std::copy( host_matrix_2.cbegin(), host_matrix_2.cend(), std::ostream_iterator< a2::node_t >( std::cout, " " ) );

    // clean up
    cudaFree( d_neighbours );
    cudaFree( d_offsets );
}

int main()
{
    using namespace csc485b;

    // Create input
    std::size_t constexpr n = 4;
    std::size_t constexpr expected_degree = n >> 1;

    //a2::edge_list_t const graph = a2::generate_graph( n, n * expected_degree );
    a2::edge_list_t const graph = a2::generate_known_graph( n, n * expected_degree );
    std::size_t const m = graph.size();

    // lazily echo out input graph
    for( auto const& e : graph )
    {
        std::cout << "(" << e.x << "," << e.y << ") ";
    }
    std::cout << std::endl;
    // allocate and memcpy input to device
    a2::edge_t * d_edges;
    cudaMalloc( (void**)&d_edges, sizeof( a2::edge_t ) * m );
    cudaMemcpyAsync( d_edges, graph.data(), sizeof( a2::edge_t ) * m, cudaMemcpyHostToDevice );

    // run your code!
    run_dense ( d_edges, n, m );
    //run_sparse( d_edges, n, m );

    return EXIT_SUCCESS;
}

In [8]:
%cuda_group_run --group "source" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"

(0,1) (1,0) (0,2) (2,0) (1,3) (3,1) (2,3) (3,2) 
Producing Dense Graph and Two Hop Reachability:
Build time: 180 us
Reachability time: 30 us
Dense Adjacency Matrix: 0 1 1 0 1 0 0 1 1 0 0 1 0 1 1 0 
Two Hop Matrix: 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 
_____________________________________________________________________________________________________________



want this for csr:<br>
neighbours_start_at = [0, 2, 4, 7]<br>
neighbours          = [2, 3, 2, 3, 0, 1, 3, 0, 1, 2] <br>
for n=8:<br>
neighbours_start_at = [0, 5, 9, 12, 18, ...]<br>
neighbours          = [2, 3, 4, 5, 6, 3, 4, 5, 7, 0, 3, 5, 0, 1, 4, 5, 6, ...]